Notebook for the GoogleDecimeterChallenge https://www.kaggle.com/competitions/smartphone-decimeter-2023


Also try to run it on google collab, works great only RAM is limited:
https://colab.research.google.com/github/Torbynator/GoogleDecimeterChallenge/blob/main/main.ipynb#scrollTo=TOn-Can4C0YP

#Only for google collab
from google.colab import drive
drive.mount('/content/drive')
!pip install  kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!kaggle competitions download -c smartphone-decimeter-2023
!unzip /content/smartphone-decimeter-2023.zip

In [1]:
#reading data
import numpy as np 
import pandas as pd
from tqdm import tqdm, trange
import math
import os

INPUT_PATH = 'sdc2023/'



In [2]:


test_input_data = []
test_gt_data = []

#iterate over all data files and store them in the respective arrays

#load test data
test_files = os.listdir(INPUT_PATH + "test")

for folder in test_files:
    smartphones = os.listdir(INPUT_PATH + "test/"+folder)
    for smartphone in smartphones:
        file =  "/device_gnss.csv"
        #store data in list while dropping first and 41st column (string data)
        test_input_data.append(pd.read_csv(INPUT_PATH + "test/" +folder+"/"+smartphone + file, usecols=[i for i in range(58) if i not in [0,40]], dtype=np.float32).to_numpy(dtype=np.float32).tolist())


In [2]:
#load train data
MAX_SAMPLES = 80
train_files = os.listdir(INPUT_PATH + "train")
sample_count=0

train_input_data = []
train_gt_data = []

for folder in train_files:
    smartphones = os.listdir(INPUT_PATH + "train/"+folder)
    for smartphone in smartphones:
        files = os.listdir(INPUT_PATH + "train/"+folder+"/"+smartphone)
        for file in files:
            if file.endswith(".csv"):
                if sample_count >= MAX_SAMPLES:
                    break   
                if "gnss" in file:
                    #store data in list while dropping first and 41st column (string data), as well as (porbably mostly) empty columns
                    #also all data points with the same timestep are seen as features of one timestep of a sample
                    train_input_data.append(pd.read_csv(INPUT_PATH + "train/" +folder+"/"+ smartphone+ "/" + file, usecols=[i for i in range(58) if i not in [0,2,3,4,23,24,25,27,28,19,29,30,31,32,33,34,35,36,40]], dtype=float).to_numpy(dtype=float).tolist())
                elif "ground_truth" in file:
                    sample_count +=1
                    #store data in list while dropping first and 2nd column (string data),(porbably mostly) empty columns
                    train_gt_data.append(pd.read_csv(INPUT_PATH + "train/"+folder+"/" + smartphone+ "/" + file,  usecols=[i for i in range(9) if i not in [0,1]], dtype=float).to_numpy(dtype=float).tolist())
                    print(f"read in {sample_count} samples")


#replace NaN values with 0
train_input_data = [[[0 if math.isnan(x) else x for x in timestep] for timestep in sample ] for sample in train_input_data]

read in 1 samples
read in 2 samples
read in 3 samples
read in 4 samples
read in 5 samples
read in 6 samples
read in 7 samples
read in 8 samples
read in 9 samples
read in 10 samples
read in 11 samples
read in 12 samples
read in 13 samples
read in 14 samples
read in 15 samples
read in 16 samples
read in 17 samples
read in 18 samples
read in 19 samples
read in 20 samples
read in 21 samples
read in 22 samples
read in 23 samples
read in 24 samples
read in 25 samples
read in 26 samples
read in 27 samples
read in 28 samples
read in 29 samples
read in 30 samples
read in 31 samples
read in 32 samples
read in 33 samples
read in 34 samples
read in 35 samples
read in 36 samples
read in 37 samples
read in 38 samples
read in 39 samples
read in 40 samples
read in 41 samples
read in 42 samples
read in 43 samples
read in 44 samples
read in 45 samples
read in 46 samples
read in 47 samples
read in 48 samples
read in 49 samples
read in 50 samples
read in 51 samples
read in 52 samples
read in 53 samples
re

In [3]:
import tensorflow as tf

#move all features with the same timestamp to one line 

for sample in range(len(train_input_data)):
    matches = 0
    print(f"sample {sample}")
    timesteps = 0
    for step in range(len(train_input_data[sample])):
        if step != 0:
            if train_input_data[sample][step-matches][0] == train_input_data[sample][step-1-matches][0]:
                train_input_data[sample][step-1-matches].extend(train_input_data[sample][step-matches])
                train_input_data[sample].pop(step-matches)
                matches +=1
            else:
                timesteps+=1

#pad input data
max_timesteps = max(max([len(sample) for sample in train_input_data]), max([len(sample) for sample in train_gt_data]))
max_features = max(max([len(feature) for sample in train_input_data for feature in sample]),max([len(feature) for sample in train_gt_data for feature in sample]))

for sample in train_input_data:
    sample.extend([[0]*max_features]*(max_timesteps-len(sample)))


train_input_data_padded = [tf.keras.preprocessing.sequence.pad_sequences(sample,value=0, padding="post", dtype=np.float64, maxlen=max_features) for sample in train_input_data]
train_input_data_padded = np.array(train_input_data_padded)

del train_input_data

#pad ground truth data
max_timesteps_gt = max_timesteps
max_features_gt = max([len(feature) for sample in train_gt_data for feature in sample])

for sample in train_gt_data:
    sample.extend([[0]*max_features_gt]*(max_timesteps_gt-len(sample)))

#convert to right data format
train_gt_data_padded = [tf.keras.preprocessing.sequence.pad_sequences(sample, value=0,padding="post", dtype=np.float64, maxlen=max_features_gt) for sample in train_gt_data]
train_gt_data_padded = np.array(train_gt_data_padded)
del train_gt_data


sample 0
sample 1
sample 2
sample 3
sample 4
sample 5
sample 6
sample 7
sample 8
sample 9
sample 10
sample 11
sample 12
sample 13
sample 14
sample 15
sample 16
sample 17
sample 18
sample 19
sample 20
sample 21
sample 22
sample 23
sample 24
sample 25
sample 26
sample 27
sample 28
sample 29
sample 30
sample 31
sample 32
sample 33
sample 34
sample 35
sample 36
sample 37
sample 38
sample 39
sample 40
sample 41
sample 42
sample 43
sample 44
sample 45
sample 46
sample 47
sample 48
sample 49
sample 50
sample 51
sample 52
sample 53
sample 54
sample 55
sample 56
sample 57
sample 58
sample 59
sample 60
sample 61
sample 62
sample 63
sample 64
sample 65
sample 66
sample 67
sample 68
sample 69
sample 70
sample 71
sample 72
sample 73
sample 74
sample 75
sample 76
sample 77
sample 78
sample 79


In [4]:
import sklearn.preprocessing as preprocess

#normalize data
scaler = preprocess.MinMaxScaler()
train_input_data_padded_normalized = []
for sample in train_input_data_padded:
    data_range = []
    train_input_data_padded_normalized.append(scaler.fit_transform(sample))
train_input_data_padded_normalized=np.array(train_input_data_padded_normalized)

train_gt_data_padded_normalized = []
for sample in train_gt_data_padded:
    data_range = []
    train_gt_data_padded_normalized.append(scaler.fit_transform(sample))
train_gt_data_padded_normalized=np.array(train_gt_data_padded_normalized)



In [15]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam

clear_session()

#create model
model = Sequential()
model.add(Masking(mask_value=0, input_shape=(train_input_data_padded.shape[1], train_input_data_padded.shape[2])))
#model.add(tf.keras.layers.BatchNormalization(axis=-1, input_shape=(train_input_data_padded.shape[1], train_input_data_padded.shape[2])))
model.add(LSTM(500, return_sequences=True))
model.add(Dense(7, activation="linear"))



model.compile(optimizer="adam", loss="MeanSquaredError")


In [16]:

print(train_input_data_padded_normalized.shape)
print(train_gt_data_padded.shape)

print(model.predict(train_input_data_padded_normalized[0:1]))
print(train_gt_data_padded_normalized[0:1])
print(model.summary())

(80, 3141, 2067)
(80, 3141, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
[[[ 0.13080561 -0.10591459 -0.02379882 ...  0.23695007 -0.16982178
    0.12289654]
  [ 0.23292425 -0.25090894 -0.02940829 ...  0.36450648 -0.22904949
    0.23885106]
  [ 0.28631237 -0.2952764  -0.05667304 ...  0.54573005 -0.37407097
    0.2750918 ]
  ...
  [ 0.3249194  -0.4081111   0.45217526 ...  0.03408051 -0.5454806
    0.21005493]
  [ 0.3249194  -0.4081111   0.45217517 ...  0.03408045 -0.5454806
    0.21005496]
  [ 0.3249194  -0.4081111   0.4521752  ...  0.03408039 -0.5454806
    0.21005493]]]
[[[0.99576724 0.00211218 0.11476906 ... 1.         0.66337243 0.99999919]
  [0.99576724 0.00211218 0.11473767 ... 1.         0.66337571 0.99999919]
  [0.99576722 0.00211218 0.11436106 ... 1.         0.66245823 0.99999919]
  ...
  [0.         1.         1.         ... 0.         0.         0.        ]
  [0.         1.         1.         ... 0.         0.         0.        ]
  [0.         1.         1.         ... 0.         

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 3141, 2067)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 3141, 500)      │     5,136,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3141, 7)        │         3,507 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,139,507 (19.61 MB)

 Trainable params: 5,139,507 (19.61 MB)

 Non-trainable params: 0 (0.00 B)

None


In [17]:

# train the model
model.fit(train_input_data_padded_normalized, train_gt_data_padded, epochs=1000, batch_size=12, validation_split=0.2)



Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 139s 23s/step - loss: 217486608225877348057088.0000 - val_loss: 149519183369527296524288.0000
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 143s 23s/step - loss: 215783689120377506627584.0000 - val_loss: 149519183369527296524288.0000
Epoch 3/1000
1/6 ━━━━━━━━━━━━━━━━━━━━ 2:08 26s/step - loss: 196675852650166985162752.0000